# Introduction 

The '01_exploration.ipynb notebook' serves as the foundation for understanding the data landscape of streaming platforms and imdb supplemental data in this project. It focuses on exploring and analyzing the structure, content, and quality of datasets collected from multiple sources.

### Exploration Goals
* Understand the structure of each dataset.
* Identify potential issues (e.g., missing values, duplicates, outliers).
* Generate summary statistics to get an overview.
* Visualize initial trends (e.g., genre distribution, ratings).

## Datasets Overview

This project utilizes five datasets from Kaggle, providing comprehensive information on popular streaming platforms and IMDb ratings. Each dataset is updated daily, ensuring accurate and relevant content.

1. **Netflix**

    * Source: [Netflix Movies & TV Series Dataset](https://www.kaggle.com/datasets/octopusteam/full-netflix-dataset)
    * **Description**: A complete collection of Netflix's available titles (movies and TV series) with IMDb-specific data such as IMDb ID, average rating, and number of votes.

2. **Apple TV+**

    * Source: [Full Apple TV+ Dataset](https://www.kaggle.com/datasets/octopusteam/full-apple-tv-dataset)
    * Description: A dataset covering all Apple TV+ titles, including key IMDb data for in-depth analysis of content quality.

3. **HBO Max**

    * Source:  [Full HBO Max Dataset](https://www.kaggle.com/datasets/octopusteam/full-hbo-max-dataset)
    * Description: An extensive collection of titles on HBO Max with associated IMDb data for comparison.

3. **Amazon Prime**

   * Source: [Full Amazon Prime Dataset](https://www.kaggle.com/datasets/octopusteam/full-amazon-prime-dataset)
    * Description: Comprehensive data on Amazon Prime's movie and TV series offerings, including IMDb-specific metrics.

4. **Hulu**

    * Source: [Full Hulu Dataset](https://www.kaggle.com/datasets/octopusteam/full-hulu-dataset)
    * Description: A dataset detailing Hulu's catalog with IMDb-related columns for evaluating content quality and popularity.

Each of the streaming platform datasets includes the following columns:

* **title**: Name of the content.
* **type**: Either "movie" or "tv series."
* **genres**: Genres associated with the title.
* **releaseYear**: Year the title was released.
* **imdbId**: Unique IMDb identifier.
* **imdbAverageRating**: Average user rating on IMDb.
* **imdbNumVotes**: Number of votes received on IMDb.
* **availableCountries**: Countries where the title is available.

Additionally, the supplementary IMDb datasets were downloaded directly from IMDB. From the initial 7 datasets available on IMDb, these datasets complemented my project and will be able to expand on the main question

[IMDb Data Files](https://datasets.imdbws.com/)

The data that I was able to extract due to its usefulness is represented in the following columns:

* title.basics.tsv.gz
    * tconst (string) - alphanumeric unique identifier of the title
    * primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
    * isAdult (boolean) - 0: non-adult title; 1: adult title
    * runtimeMinutes – primary runtime of the title, in minutes
    * genres (string array) – up to three genres associated with the title
      
* title.ratings.tsv.gz
  
    * tconst (string) - alphanumeric unique identifier of the title
    * averageRating – weighted average of all the individual user ratings
    * numVotes - number of votes the title has received

This initial exploration will set the stage for more advanced analyses, such as clustering, statistical comparisons, and the evaluation of platform value.

### Imports

In [1]:
# Import Libraries

import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys
sys.path.append(r"C:\Users\kimbe\Documents\StreamingAnalysis\scripts")  # Corrected path
# Set to show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


In [3]:
# Import custom scripts

from utils import *
from explore import *


In [4]:
# Define the base path
base_path = r"C:\Users\kimbe\Documents\StreamingAnalysis\data\raw_data"

# Load the datasets
amazon_df = pd.read_csv(f'{base_path}\\amazon_catalog.csv')
hulu_df = pd.read_csv(f'{base_path}\\hulu_catalog.csv')
netflix_df = pd.read_csv(f'{base_path}\\netflix_catalog.csv')
hbo_df = pd.read_csv(f'{base_path}\\hbo_catalog.csv')
apple_df = pd.read_csv(f'{base_path}\\apple_catalog.csv')

# List of datasets
datasets = [amazon_df, hulu_df, netflix_df, hbo_df, apple_df]
platforms = ["Amazon", "Hulu", "Netflix", "HBO", "Apple"]


In [5]:
# Count and remove rows that are not available in the US
cleaned_dfs = process_platform_data(datasets, platforms)


## Clean IMDb datasets

In [6]:
# Load the basics and ratings datasets
basics_df = pd.read_csv(os.path.join(base_path, 'imdb_basics.csv'), na_values='\\N', low_memory=False)
ratings_df = pd.read_csv(os.path.join(base_path, 'imdb_ratings.csv'), na_values='\\N', low_memory=False)

# Print the column names of both datasets
print("Basics Columns:", basics_df.columns)
print("Ratings Columns:", ratings_df.columns)



Basics Columns: Index(['tconst', 'titleType', 'primaryTitle', 'originalTitle', 'isAdult',
       'startYear', 'endYear', 'runtimeMinutes', 'genres'],
      dtype='object')
Ratings Columns: Index(['tconst', 'averageRating', 'numVotes'], dtype='object')


In [7]:
# Combine and rename
imdb_info = pd.merge(basics_df, ratings_df, on='tconst', how='inner')
imdb_info.rename(columns={"tconst": "imdbId", "primaryTitle": "title", "titleType": "type"}, inplace=True)
imdb_info.head()

,imdbId,type,title,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0.0,1894.0,NaN,1,"Documentary,Short",5.7,2106
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0.0,1892.0,NaN,5,"Animation,Short",5.6,283
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0.0,1892.0,NaN,5,"Animation,Comedy,Romance",6.5,2128
3,tt0000004,short,Un bon bock,Un bon bock,0.0,1892.0,NaN,12,"Animation,Short",5.4,182
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0.0,1893.0,NaN,1,"Comedy,Short",6.2,2859


In [8]:
missing_values = imdb_info.isnull().sum()
missing_values

imdbId                  0
type                    0
title                   1
originalTitle           1
isAdult                 0
startYear             246
endYear           1448495
runtimeMinutes     450199
genres              21466
averageRating           0
numVotes                0
dtype: int64

In [9]:
# Drop isAdult as it is inaccurate.
imdb_info = imdb_info.drop(columns=['isAdult'])

# Drop endYear 
imdb_info = imdb_info.drop(columns=['endYear'])

# Drop originalTitle
imdb_info = imdb_info.drop(columns=['originalTitle'])

# Drop runTime
imdb_info = imdb_info.drop(columns=['runtimeMinutes'])

# Drop startYear
imdb_info = imdb_info.drop(columns=['startYear'])


## Platform Dataset Exploration

In [10]:
# Count the number of rows missing IMDbID and remove them
missing_percent_df, cleaned_datasets = calculate_missing_percent_and_clean(datasets, platforms)

# Display the missing percentages
missing_percent_df

,Platform,Missing Percent
4,Apple,7.829413
0,Amazon,5.839357
1,Hulu,4.776848
3,HBO,4.560379
2,Netflix,3.585823


In [11]:
# Apply the function to all platforms datasets
datasets = drop_duplicates_by_imdbId(datasets)

# Check the number of rows before and after dropping duplicates
for df in datasets:
    print(f"Rows after dropping duplicates: {df.shape[0]}")

Rows after dropping duplicates: 23187
Rows after dropping duplicates: 2731
Rows after dropping duplicates: 6936
Rows after dropping duplicates: 2616
Rows after dropping duplicates: 11325


### Check Basic Structure

In [12]:
# Generate the separate summaries
separate_summaries = dataset_summary_separate(datasets, platforms)

# Access individual tables
missing_values_table = separate_summaries["Missing Values"]
data_types_table = separate_summaries["Data Type"]
unique_values_table = separate_summaries["Unique Values"]
duplicates_table = separate_summaries["Duplicates"]

#### Missing Values

In [13]:
# How many missing values each column contains for every dataset.
print("----- Missing Values -----")
missing_values_table

----- Missing Values -----


,Amazon,Hulu,Netflix,HBO,Apple
Column,,,,,
title,0,0,0,0,0
type,0,0,0,0,0
genres,16,1,1,6,121
releaseYear,2,0,0,1,1
imdbId,0,0,0,0,0
imdbAverageRating,441,18,34,27,313
imdbNumVotes,441,18,34,27,313
availableCountries,0,0,0,0,0


In [14]:
# Remove rows missing 'releaseYear'
cleaned_datasets = []
for platform, df in zip(platforms, datasets):
    print(f"--- {platform} Dataset ---")
    print(f"Rows before removal: {df.shape[0]}")
    df_cleaned = df.dropna(subset=['releaseYear'])
    cleaned_datasets.append(df_cleaned)
    print(f"Rows after removal: {df_cleaned.shape[0]}\n")

# Update the datasets list with cleaned datasets
datasets = cleaned_datasets

--- Amazon Dataset ---
Rows before removal: 23187
Rows after removal: 23185

--- Hulu Dataset ---
Rows before removal: 2731
Rows after removal: 2731

--- Netflix Dataset ---
Rows before removal: 6936
Rows after removal: 6936

--- HBO Dataset ---
Rows before removal: 2616
Rows after removal: 2615

--- Apple Dataset ---
Rows before removal: 11325
Rows after removal: 11324



In [15]:
# Copy of df
amazon_df = amazon_df.copy()
hulu_df = hulu_df.copy()
netflix_df = netflix_df.copy()
hbo_df = hbo_df.copy()
apple_df = apple_df.copy()

#List of columns to drop 
columns_to_drop = ['imdbAverageRating', 'imdbNumVotes']  

# Apply the function to all platform datasets
datasets = drop_columns_from_datasets(datasets, columns_to_drop)

C:\Users\kimbe\Documents\StreamingAnalysis\scripts\explore.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=columns_to_drop, inplace=True, errors='ignore')


In [16]:
# Reprint the missing values table after cleaning
missing_values_table = print_missing_values(datasets, platforms)
missing_values_table

----- Missing Values -----


,Amazon,Hulu,Netflix,HBO,Apple
title,0,0,0,0,0
type,0,0,0,0,0
genres,16,1,1,6,121
releaseYear,0,0,0,0,0
imdbId,0,0,0,0,0
availableCountries,0,0,0,0,0


In [17]:
# Number of unique values in each column for every dataset.
print("----- Unique Values -----")
unique_values_table

----- Unique Values -----


,Amazon,Hulu,Netflix,HBO,Apple
Column,,,,,
title,22324,2713,6832,2592,11071
type,2,2,2,2,2
genres,809,374,484,354,618
releaseYear,112,55,59,115,93
imdbId,23187,2731,6936,2616,11325
imdbAverageRating,89,71,79,64,77
imdbNumVotes,6747,2212,4823,2088,5339
availableCountries,4600,2,2763,382,169


In [18]:
# Data type of each column for every dataset
print("----- Data Types -----")
data_types_table

----- Data Types -----


,Amazon,Hulu,Netflix,HBO,Apple
Column,,,,,
title,object,object,object,object,object
type,object,object,object,object,object
genres,object,object,object,object,object
releaseYear,float64,float64,float64,float64,float64
imdbId,object,object,object,object,object
imdbAverageRating,float64,float64,float64,float64,float64
imdbNumVotes,float64,float64,float64,float64,float64
availableCountries,object,object,object,object,object


In [19]:
# Number of duplicate rows per column for every dataset
print("----- Duplicate Values -----")
duplicates_table

----- Duplicate Values -----


,Amazon,Hulu,Netflix,HBO,Apple
Column,,,,,
title,863,18,104,24,254
type,23185,2729,6934,2614,11323
genres,22377,2356,6451,2261,10706
releaseYear,23074,2676,6877,2500,11231
imdbId,0,0,0,0,0
imdbAverageRating,23097,2659,6856,2551,11247
imdbNumVotes,16439,518,2112,527,5985
availableCountries,18587,2729,4173,2234,11156


### Filter IMDb datasets so we can use them to fill missing values

In [20]:
# Extract imbdId from platforms and create a df of imdbid with only those
platform_imdb_ids = {
    "Amazon": amazon_df["imdbId"].unique(),
    "Hulu": hulu_df["imdbId"].unique(),
    "Netflix": netflix_df["imdbId"].unique(),
    "HBO": hbo_df["imdbId"].unique(),
    "Apple": apple_df["imdbId"].unique()
}

# Combine the imdbIds from all platforms into a single list (unique values only)
all_platform_imdb_ids = set()
for platform in platform_imdb_ids:
    all_platform_imdb_ids.update(platform_imdb_ids[platform])

# Filter the combined IMDb DataFrame to include only the imdbIds that are in the platforms
filtered_imdb_info= imdb_info[imdb_info['imdbId'].isin(all_platform_imdb_ids)]

# Display the resulting DataFrame
print(f"Filtered IMDb DataFrame Shape: {filtered_imdb_info.shape}")
filtered_imdb_info.head()

Filtered IMDb DataFrame Shape: (43745, 6)


,imdbId,type,title,genres,averageRating,numVotes
300,tt0000417,short,A Trip to the Moon,"Adventure,Comedy,Fantasy",8.1,57373
337,tt0000499,short,An Impossible Voyage,"Action,Adventure,Family",7.5,4153
1683,tt0004873,movie,Alice in Wonderland,"Adventure,Family,Fantasy",6.1,805
1735,tt0005078,movie,The Cheat,"Drama,Romance",6.5,2893
1777,tt0005302,movie,"Fanchon, the Cricket",Drama,6.4,366


In [21]:
# Number of unique values in each column for every dataset.
print("----- Unique Values -----")
unique_values_table

----- Unique Values -----


,Amazon,Hulu,Netflix,HBO,Apple
Column,,,,,
title,22324,2713,6832,2592,11071
type,2,2,2,2,2
genres,809,374,484,354,618
releaseYear,112,55,59,115,93
imdbId,23187,2731,6936,2616,11325
imdbAverageRating,89,71,79,64,77
imdbNumVotes,6747,2212,4823,2088,5339
availableCountries,4600,2,2763,382,169


In [22]:
missing_values = filtered_imdb_info.isnull().sum()
missing_values

imdbId             0
type               0
title              0
genres           312
averageRating      0
numVotes           0
dtype: int64

In [23]:
# Reprint the missing values table after cleaning
missing_values_table = print_missing_values(datasets, platforms)
missing_values_table

----- Missing Values -----


,Amazon,Hulu,Netflix,HBO,Apple
title,0,0,0,0,0
type,0,0,0,0,0
genres,16,1,1,6,121
releaseYear,0,0,0,0,0
imdbId,0,0,0,0,0
availableCountries,0,0,0,0,0


In [24]:
# Check the minimum and maximum of IMDb's average ratings
print(filtered_imdb_info['averageRating'].min())
print(filtered_imdb_info['averageRating'].max())


1.1
9.9


In [25]:
# Platform Info
platforms = ['Amazon', 'Hulu', 'Netflix', 'HBO', 'Apple']
platform_dfs = [amazon_df, hulu_df, netflix_df, hbo_df, apple_df]

# Apply the function for each platform dataset
for platform_name, df in zip(platforms, platform_dfs):
    add_platform_column(df, platform_name)

## Merging Platforms

In [26]:
# Merge the datasets
merged_platforms_df = merge_platform_datasets(platforms, platform_dfs)

In [27]:
merged_platforms_df.head()

,imdbId,type,genres,releaseYear,title,Amazon,Hulu,Netflix,HBO,Apple
0,tt0000417,movie,"Action, Adventure, Comedy",1902.0,A Trip to the Moon,0,0,0,1,0
1,tt0000499,movie,"Action, Adventure, Family",1904.0,An Impossible Voyage,0,0,0,1,0
2,tt0004873,movie,"Adventure, Family, Fantasy",1915.0,Alice in Wonderland,1,0,0,0,0
3,tt0005078,movie,"Drama, Romance",1915.0,The Cheat,1,0,0,0,0
4,tt0005302,movie,Drama,1915.0,"Fanchon, the Cricket",1,0,0,0,0


In [28]:
# Change datatype of releaseYear
# Convert releaseYear to int while handling NaN values
merged_platforms_df['releaseYear'] = merged_platforms_df['releaseYear'].fillna(0).astype(int)
streaming_platforms = merged_platforms_df

# Final output after cleaning

In [29]:
streaming_platforms.head(10)

,imdbId,type,genres,releaseYear,title,Amazon,Hulu,Netflix,HBO,Apple
0,tt0000417,movie,"Action, Adventure, Comedy",1902,A Trip to the Moon,0,0,0,1,0
1,tt0000499,movie,"Action, Adventure, Family",1904,An Impossible Voyage,0,0,0,1,0
2,tt0004873,movie,"Adventure, Family, Fantasy",1915,Alice in Wonderland,1,0,0,0,0
3,tt0005078,movie,"Drama, Romance",1915,The Cheat,1,0,0,0,0
4,tt0005302,movie,Drama,1915,"Fanchon, the Cricket",1,0,0,0,0
5,tt0005339,movie,Drama,1915,A Fool There Was,1,0,0,0,0
6,tt0005812,movie,"Comedy, Short",1915,A Night in the Show,0,0,0,1,0
7,tt0006327,movie,"Comedy, Romance",1915,Young Romance,1,0,0,0,0
8,tt0006864,movie,"Drama, History",1916,Intolerance,1,0,0,0,0
9,tt0007264,movie,"Comedy, Short",1916,The Rink,0,0,0,1,0


In [30]:
filtered_imdb_info.head(10)

,imdbId,type,title,genres,averageRating,numVotes
300,tt0000417,short,A Trip to the Moon,"Adventure,Comedy,Fantasy",8.1,57373
337,tt0000499,short,An Impossible Voyage,"Action,Adventure,Family",7.5,4153
1683,tt0004873,movie,Alice in Wonderland,"Adventure,Family,Fantasy",6.1,805
1735,tt0005078,movie,The Cheat,"Drama,Romance",6.5,2893
1777,tt0005302,movie,"Fanchon, the Cricket",Drama,6.4,366
1793,tt0005339,movie,A Fool There Was,Drama,5.7,1080
1921,tt0005812,short,A Night in the Show,"Comedy,Short",6.4,2464
2030,tt0006327,movie,Young Romance,"Comedy,Romance",5.8,223
2191,tt0006864,movie,Intolerance,"Drama,History",7.7,17098
2280,tt0007264,short,The Rink,"Comedy,Short",7.0,4410


In [31]:
streaming_platforms.describe()

,releaseYear,Amazon,Hulu,Netflix,HBO,Apple
count,44568.000000,44568.000000,44568.000000,44568.000000,44568.000000,44568.000000
mean,2008.458019,0.520261,0.061277,0.155627,0.058697,0.254106
std,24.923968,0.499595,0.239841,0.362506,0.235059,0.435362
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2005.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2016.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,2020.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,2025.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [32]:
filtered_imdb_info.describe()

,averageRating,numVotes
count,43745.000000,4.374500e+04
mean,6.143091,1.553606e+04
std,1.309902,7.629059e+04
min,1.100000,5.000000e+00
25%,5.300000,1.750000e+02
50%,6.300000,7.860000e+02
75%,7.100000,4.167000e+03
max,9.900000,2.951998e+06


In [33]:
null_counts = streaming_platforms.isnull().sum()
print(null_counts)

imdbId           0
type             0
genres         144
releaseYear      0
title            0
Amazon           0
Hulu             0
Netflix          0
HBO              0
Apple            0
dtype: int64


In [34]:
null_counts = filtered_imdb_info.isnull().sum()
print(null_counts)

imdbId             0
type               0
title              0
genres           312
averageRating      0
numVotes           0
dtype: int64


In [35]:
print(filtered_imdb_info.columns)
print(streaming_platforms.columns)


Index(['imdbId', 'type', 'title', 'genres', 'averageRating', 'numVotes'], dtype='object')
Index(['imdbId', 'type', 'genres', 'releaseYear', 'title', 'Amazon', 'Hulu',
       'Netflix', 'HBO', 'Apple'],
      dtype='object')


In [36]:
# Create a copy of filtered_imdb_info to avoid the SettingWithCopyWarning
filtered_imdb_info_copy = filtered_imdb_info.copy()

# Rename 'genres' column in the copy of filtered_imdb_info
filtered_imdb_info_copy.rename(columns={'genres': 'imdb_genres'}, inplace=True)

# Merge streaming_platforms with the modified filtered_imdb_info copy
final_df = pd.merge(streaming_platforms, filtered_imdb_info_copy[['imdbId', 'imdb_genres', 'averageRating', 'numVotes']], 
                     on='imdbId', how='left')

# Fill missing 'genres' in streaming_platforms with 'imdb_genres' from filtered_imdb_info
final_df['genres'] = final_df['genres'].fillna(final_df['imdb_genres'])

# Drop the temporary 'imdb_genres' column
final_df.drop(columns=['imdb_genres'], inplace=True)

streaming_titles_info = final_df



In [37]:
# Specify the file path
file_path = r'C:\Users\kimbe\Documents\StreamingAnalysis\data\cleaned_data\streaming_titles_info.csv'

# Save the final merged DataFrame to the specified path
streaming_titles_info.to_csv(file_path, index=False)